In [ ]:
! pip -q install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 81.1 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 1.22.4
    Uninstalling numpy-1.22.4:
      Successfully uninstalled numpy-1.22.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
numba 0.56.4 requires numpy<1.24,>=1.18, but you have numpy 1.24.3 which is incompatible.
tensorflow 2.12.0 requires numpy<1.24,>=1.22, but you have numpy 1.24.3 which is incompatible.


In [ ]:
import glob
import logging
import os
import pickle
import random
import re
import shutil
from typing import Dict, List, Tuple

import pandas as pd
import numpy as np
import torch

from sklearn.model_selection import train_test_split

from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import DataLoader, Dataset, RandomSampler, SequentialSampler
from torch.utils.data.distributed import DistributedSampler
from tqdm.notebook import tqdm, trange

from pathlib import Path

from transformers import (
    MODEL_WITH_LM_HEAD_MAPPING,
    WEIGHTS_NAME,
    AdamW,
    AutoConfig,
    AutoModelWithLMHead,
    AutoModelForCausalLM,
    AutoTokenizer,
    PreTrainedModel,
    PreTrainedTokenizer,
    get_linear_schedule_with_warmup,
)

#piattaforma di visualizzazione di dati sviluppata da Google per la visualizzazione di dati in tempo reale e per il machine learning. 
try:
    from torch.utils.tensorboard import SummaryWriter
except ImportError:
    from tensorboardX import SummaryWriter

#definite le costanti MODEL_CONFIG_CLASSES e MODEL_TYPES utilizzate per selezionare il tipo di modello e configurazione da utilizzare. Viene anche creato un logger per registrare eventi e messaggi nel codice.
logger = logging.getLogger(__name__)

MODEL_CONFIG_CLASSES = list(MODEL_WITH_LM_HEAD_MAPPING.keys())
MODEL_TYPES = tuple(conf.model_type for conf in MODEL_CONFIG_CLASSES)

In [ ]:
class Args():
    def __init__(self):
        self.output_dir = 'output'
        self.model_type = 'gpt2'
        self.model_name_or_path = 'microsoft/DialoGPT-small'
        self.config_name = 'microsoft/DialoGPT-small'
        self.tokenizer_name = 'microsoft/DialoGPT-small'
        self.cache_dir = 'cached'
        self.block_size = 512
        self.do_train = True
        self.do_eval = False
        self.evaluate_during_training = False
        self.per_gpu_train_batch_size = 4
        self.per_gpu_eval_batch_size = 4
        self.gradient_accumulation_steps = 1
        self.learning_rate = 5e-5
        self.weight_decay = 0.0
        self.adam_epsilon = 1e-8
        self.max_grad_norm = 1.0
        self.num_train_epochs = 1
        self.max_steps = -1
        self.warmup_steps = 0
        self.save_steps = 3500
        self.save_total_limit = None
        self.no_cuda = False
        self.overwrite_output_dir = True
        self.overwrite_cache = True
        self.seed = 42
        self.local_rank = -1

args = Args()

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
df = pd.read_csv('/content/drive/MyDrive/en_conv_small.csv')
print(len(df))
print(df.columns)
trn_df, val_df = train_test_split(df, test_size = 0.2)

Mounted at /content/drive
50468
Index(['response', 'context', 'context/0', 'context/1', 'context/2',
       'context/3', 'context/4', 'context/5', 'context/6', 'context/7',
       'context/8', 'context/9'],
      dtype='object')
25068


In [ ]:
#La funzione from_pretrained viene utilizzata per caricare un tokenizer già addestrato sulla base del parametro model_name_or_path, che indica il nome del modello o il percorso in cui è stato salvato il tokenizer. 
#Il parametro cache_dir viene utilizzato per specificare la cartella in cui memorizzare eventuali cache per velocizzare il caricamento dei dati.
tokenizer = AutoTokenizer.from_pretrained(args.model_name_or_path, cache_dir=args.cache_dir)

In [ ]:
#processing del dataset, ogni riga(conversazione) viene convertita in embedding ids, e ogni frase viene conclusa con il end of string token, la singola conversazione viene poi concatenata in una sola frase
#infine il dataset viene convertito in tensore(sarà poi successivamente convertito in dataloader per essere dato in input al modello)
def construct_conv(row, tokenizer, eos = True):
    flatten = lambda l: [item for sublist in l for item in sublist]
    conv = list(reversed([tokenizer.encode(x) + [tokenizer.eos_token_id] for x in row]))
    conv = flatten(conv)
    return conv

class ConversationDataset(Dataset):
    def __init__(self, tokenizer: PreTrainedTokenizer, args, df, block_size=512):

        block_size = block_size - (tokenizer.model_max_length - tokenizer.max_len_single_sentence)

        self.examples = []
        for _, row in df.iterrows():
            conv = construct_conv(row, tokenizer)
            if len(conv) > block_size: continue
            self.examples.append(conv)

    def __len__(self):
        return len(self.examples)

    def __getitem__(self, item):
        return torch.tensor(self.examples[item], dtype=torch.long)

In [ ]:
def load_examples(args, tokenizer, df_trn, df_val, evaluate=False):
    return ConversationDataset(tokenizer, args, df_val if evaluate else df_trn)

def set_seed(args):
    random.seed(args.seed)
    np.random.seed(args.seed)
    torch.manual_seed(args.seed)
    if args.n_gpu > 0:
        torch.cuda.manual_seed_all(args.seed)

In [ ]:
# Training of model
def train(args, train_dataset, model: PreTrainedModel, tokenizer: PreTrainedTokenizer) -> Tuple[int, float]:    

    #definizione del batch size di training, che dipende dal numero di GPU utilizzate per l'addestramento.
    args.train_batch_size = args.per_gpu_train_batch_size * max(1, args.n_gpu)

    #creare un batch di input a partire da una lista di tensori di esempio, in modo che i tensori abbiano le stesse dimensioni all'interno del batch.
    #utilizza il metodo pad_sequence della libreria PyTorch per aggiungere padding ai tensori degli esempi più corti per renderli della stessa lunghezza di quello più lungo all'interno del batch. 
    #In particolare, se il tokenizer utilizzato ha un token di padding (_pad_token), questo viene utilizzato per effettuare il padding dei tensori degli esempi più corti. 
    #Altrimenti, viene utilizzato il valore di default fornito da PyTorch (padding_value=0).
    def collate(examples: List[torch.Tensor]):
        if tokenizer._pad_token is None:
            return pad_sequence(examples, batch_first=True)
        return pad_sequence(examples, batch_first=True, padding_value=tokenizer.pad_token_id)

    #campionatore casuale seleziona casualmente gli elementi dall'insieme di dati in modo che ogni epoca di addestramento possa vedere gli elementi in ordine casuale, garantendo che il modello non memorizzi l'ordine degli elementi. 
    #Questo è importante perché se il modello memorizza l'ordine degli elementi, potrebbe non generalizzare bene quando si incontrano nuovi dati.
    train_sampler = RandomSampler(train_dataset) if args.local_rank == -1 else DistributedSampler(train_dataset)
    #inizializza il dataloader per il set di dati di addestramento. Il dataloader carica i dati dal set di dati in batch, per consentire l'addestramento del modello in modo efficiente.
    train_dataloader = DataLoader(
        train_dataset, sampler=train_sampler, batch_size=args.train_batch_size, collate_fn=collate, drop_last = True
    )

    
    #t_total viene impostato sul numero totale di batch nel dataloader di addestramento (len(train_dataloader)) moltiplicato per il numero di epoche (args.num_train_epochs).
    t_total = len(train_dataloader) * args.num_train_epochs

    #Questa parte di codice si occupa di adattare la dimensione della matrice di embedding del modello al numero di token presenti nel tokenizer.
    model.resize_token_embeddings(len(tokenizer))
    # add_special_tokens_(model, tokenizer)

    # Prepare optimizer and schedule (linear warmup and decay)
    #definisce un'istanza di un ottimizzatore AdamW e di un scheduler di apprendimento, entrambi utilizzati per l'addestramento del modello.
    
    no_decay = ["bias", "LayerNorm.weight"]#parametri dei modelli che non devono essere soggetti a decay durante la discesa del gradiente.
    # raggruppa i parametri del modello in base a se necessitano o meno di weight decay.
    #il weight decay è una tecnica di regolarizzazione utilizzata per prevenire l'overfitting del modello durante l'addestramento. 
    #consiste nell'aggiungere una penalità alla funzione di loss in base alla norma L2 dei pesi del modello.
    optimizer_grouped_parameters = [
        {
            "params": [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)],
            "weight_decay": args.weight_decay,
        },
        {"params": [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], "weight_decay": 0.0},
    ]
    optimizer = torch.optim.AdamW(optimizer_grouped_parameters, lr=args.learning_rate, eps=args.adam_epsilon)#algoritmo di ottimizzazione basato su Adam che applica weight decay solo sui parametri che richiedono l'applicazione di questo tipo di regolarizzazione.
    # definisce uno scheduler di tipo lineare con warmup, che aumenta gradualmente il tasso di apprendimento nei primi num_warmup_steps passi di addestramento e lo riduce linearmente fino a 0 in corrispondenza dell'ultimo passo.
    scheduler = get_linear_schedule_with_warmup(
        optimizer, num_warmup_steps=args.warmup_steps, num_training_steps=t_total
    )

    #stampa alcune informazioni utili per l'addestramento del modello.
    logger.info("***** Running training *****")
    logger.info("  Num examples = %d", len(train_dataset))
    logger.info("  Num Epochs = %d", args.num_train_epochs)
    logger.info("  Instantaneous batch size per GPU = %d", args.per_gpu_train_batch_size)
    logger.info("  Gradient Accumulation steps = %d", args.gradient_accumulation_steps)
    logger.info("  Total optimization steps = %d", t_total)

    global_step = 0
    epochs_trained = 0
    steps_trained_in_current_epoch = 0

    tr_loss = 0.0
    # serve per azzerare i gradienti dei parametri del modello prima di eseguire il passaggio all'indietro (backward pass) durante l'addestramento. 
    #Questo è necessario perché PyTorch accumula i gradienti durante il backward pass, quindi è necessario azzerarli manualmente prima di eseguire un nuovo passaggio all'indietro.
    model.zero_grad()
    #L'oggetto trange() fornisce una barra di avanzamento che indica il numero di epoche che sono state completate. L'argomento desc serve per specificare una descrizione della barra di avanzamento. 
    #Il parametro disable viene utilizzato per disabilitare la barra di avanzamento se il processo non è rank 0 o se la variabile args.local_rank non è presente nel sistema.
    train_iterator = trange(
        epochs_trained, int(args.num_train_epochs), desc="Epoch", disable=args.local_rank not in [-1, 0]
    )

    set_seed(args)  # riproducibilità

    #training starts
    for _ in train_iterator:#ciclo che scorre attraverso il numero di epoche
        #
        epoch_iterator = tqdm(train_dataloader, desc="Iteration", disable=args.local_rank not in [-1, 0])
        for step, batch in enumerate(epoch_iterator):#ciclo che scorre attraverso i dati di addestramento batch


            #Prepara i dati per l'addestramento del modello. Se la lunghezza dei dati di input è superiore a 1024, il batch viene saltato.
            inputs, labels = (batch, batch)
            if inputs.shape[1] > 1024: continue
            #Carica i dati sulla GPU se disponibile.
            inputs = inputs.to(args.device)
            labels = labels.to(args.device)
            #Imposta il modello in modalità di addestramento.
            model.train()
            #Passa i dati di input al modello e ottiene l'output.
            outputs = model(inputs, labels=labels)
            #Calcola la perdita del modello (loss) in base all'output.
            loss = outputs[0]  # model outputs are always tuple in transformers (see doc)

            #esegue la retropropagazione del gradiente
            loss.backward()

            #Aggiunge la perdita corrente (loss.item()) alla perdita totale (tr_loss).
            tr_loss += loss.item()

            #Se il numero di passi di accumulazione del gradiente è stato raggiunto (args.gradient_accumulation_steps), esegue l'ottimizzazione del modello.
            if (step + 1) % args.gradient_accumulation_steps == 0:
                
                torch.nn.utils.clip_grad_norm_(model.parameters(), args.max_grad_norm)
                optimizer.step()
                #Aggiorna il programma di ottimizzazione per il modello tramite la funzione scheduler.step().
                scheduler.step()  # Update learning rate schedule
                #Azzera i gradienti
                model.zero_grad()
                #Aggiorna il conteggio dei passi globali (global_step).
                global_step += 1


            #Se il numero massimo di passi di addestramento (t_total) è stato raggiunto, termina l'addestramento.
            if global_step > t_total:
                checkpoint_prefix = "checkpoint"
                output_dir = os.path.join(args.output_dir, "{}-{}".format(checkpoint_prefix, global_step))
                os.makedirs(output_dir, exist_ok=True)
                tokenizer.save_pretrained(output_dir)
                model.save_pretrained(output_dir)
                epoch_iterator.close()
                train_iterator.close()
                break
        
    
    #Restituisce il conteggio globale dei passi (global_step) e la perdita totale dell'addestramento (tr_loss / global_step).
    return global_step, tr_loss / global_step

In [ ]:
def main(df_trn, df_val):
    args = Args()
    
    #Successivamente, viene controllato se l'output directory specificata dall'utente esiste già e contiene dei file, se l'utente ha richiesto di addestrare il modello, 
    #se non ha specificato di sovrascrivere l'output directory e se non ha richiesto di continuare l'addestramento
    if (
        os.path.exists(args.output_dir)
        and os.listdir(args.output_dir)
        and args.do_train
        and not args.overwrite_output_dir
        and not args.should_continue
    ):
        raise ValueError(
            "Output directory ({}) already exists and is not empty. Use --overwrite_output_dir to overcome.".format(
                args.output_dir
            )
        )

    #Vengono poi configurati CUDA, la GPU e l'addestramento distribuito, se possibile. Viene inoltre configurato il logging per registrare informazioni sul processo di addestramento.
    #Viene infine caricato il modello, il tokenizer e la configurazione associata, e se richiesto viene avviato 
    #l'addestramento, viene salvato il modello e i relativi dati e infine viene effettuata l'evaluazione dei risultati. I risultati dell'evaluazione vengono restituiti dalla funzione.

    device = torch.device("cuda")
    args.n_gpu = torch.cuda.device_count()
    args.device = device

    set_seed(args)

    #Questo codice carica un modello di linguaggio preaddestrato e le relative configurazioni e tokenizer
    config = AutoConfig.from_pretrained(args.config_name, cache_dir=args.cache_dir)
    tokenizer = AutoTokenizer.from_pretrained(args.tokenizer_name, cache_dir=args.cache_dir)
    model = AutoModelForCausalLM.from_pretrained(
        args.model_name_or_path,
        from_tf=False,
        config=config,
        cache_dir=args.cache_dir,
    )
    model.to(args.device)
    
    logger.info("Training parameters %s", args)

    # Training
    if args.do_train:
        train_dataset = load_examples(args, tokenizer, df_trn, df_val, evaluate=False)

        #inizia il training vero e proprio
        global_step, tr_loss = train(args, train_dataset, model, tokenizer)
        logger.info(" global_step = %s, average loss = %s", global_step, tr_loss)

    #salva il modello e il tokenizer, insieme ai relativi parametri di configurazione in args.output_dir, che è la directory di output specificata nella linea 2.
    #In particolare, il modello addestrato viene salvato usando il metodo save_pretrained(), il tokenizer viene salvato usando il metodo save_pretrained(), 
    #mentre i parametri di addestramento vengono salvati usando la funzione torch.save().
    #Inoltre, il modello e il tokenizer vengono caricati nuovamente dalla directory specificata per verificare che il salvataggio sia stato effettuato correttamente.
    if args.do_train:
        # Create output directory if needed
        os.makedirs(args.output_dir, exist_ok=True)

        logger.info("Saving model checkpoint to %s", args.output_dir)
        #salva il modello addestrato, per ricaricarlo dopo
        model.save_pretrained(args.output_dir)
        tokenizer.save_pretrained(args.output_dir)

        
        torch.save(args, os.path.join(args.output_dir, "training_args.bin"))

        # carica il modello fine tuned
        model = AutoModelForCausalLM.from_pretrained(args.output_dir)
        tokenizer = AutoTokenizer.from_pretrained(args.output_dir)
        model.to(args.device)


In [ ]:
main(trn_df, val_df)

/usr/local/lib/python3.10/dist-packages/transformers/models/auto/modeling_auto.py:1352: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  warnings.warn(


/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:407: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

Iteration:   0%|          | 0/5013 [00:00<?, ?it/s]

In [ ]:
tokenizer = AutoTokenizer.from_pretrained('microsoft/DialoGPT-small')
model = AutoModelWithLMHead.from_pretrained('output')
#input viene codificato in token utilizzando il tokenizer fornito. Successivamente, l'input utente viene concatenato alla storia della chat (inizialmente vuota) e 
#passato al modello di generazione del linguaggio per ottenere una risposta. La risposta viene quindi decodificata dal token in testo e stampata a schermo.
step=0
while True:
    # tokenizza l'user input, aggiunge eos token e restituisce tensore
    new_user_input_ids = tokenizer.encode(input(">> User:") + tokenizer.eos_token, return_tensors='pt')
    
    # print(new_user_input_ids)

    # aggiunge l'input dell'utente alla storia della conversazione(se l'input non è il primo)
    bot_input_ids = torch.cat([chat_history_ids, new_user_input_ids], dim=-1) if step > 0 else new_user_input_ids
    step+=1
    chat_history_ids = model.generate(
        bot_input_ids, max_length=1000,
        pad_token_id=tokenizer.eos_token_id,
        top_p=0.92, top_k = 50
    )
    
    # stampa l'output generato dal modello
    print("OUTPUT: {}".format(tokenizer.decode(chat_history_ids[:, bot_input_ids.shape[-1]:][0], skip_special_tokens=True)))

KeyboardInterrupt: ignored